# Network Operations

In this notebook we will assemble our project.  We will explore different functions on our dataset and compile them into a workflow ready for production.

The functions we will use will be a mix of `hub` based functions from our [MLRun Functions](http://github.com/mlrun/functions) repo, local and git based notebooks.

> The notebook should be run after generating the data in the [Generator Notebook](generator.ipynb)


we will start by setting up our environment, Loading MLRun and some utilities we will need

In [1]:
# Utils
import os
import json
import urllib
import numpy as np

# MLRun imports
from mlrun import mlconf

# Setup API Endpoint
mlconf.dbpath = 'http://mlrun-api:8080'

> If you are using another version of our `hub://` please set it up in the following cell.
* The url can parse {name} and {tag} to the given url

In [2]:
# Set Hub URL address if using a local version
# mlconf.hub_url = '/User/functions/{name}/function.yaml'

Now lets define our current project 

## Create a project from a git repository

In [3]:
from mlrun import new_project

# update the dir and repo to reflect real locations 
# the remote git repo must be initialized in GitHub
project_dir = os.path.abspath('./')
remote_git = 'https://github.com/mlrun/demo-network-operations.git'

# Create the project
newproj = new_project('network-operations', project_dir, init_git=True)

# We can update our project directory to the latest status by running
# newproj.pull()

Now that we have our project directory, lets forword our artifacts there to keep track of them

In [4]:
# Define an artifact path to keep track of where our artifacts are going
ARTIFACT_PATH =  os.path.join(os.path.abspath(newproj.context), 'artifacts')
mlconf.artifact_path = ARTIFACT_PATH

## Create and run functions

As we receive a new dataset, the first thing we would like to do is to explore it a bit, we can do that using our `describe` function in `mlrun/functions`

In [5]:
from mlrun import mount_v3io, new_model_server

In [6]:
# Import the functions
# Functions From hub
tag = 'development'
newproj.set_function(func=f'hub://aggregate:{tag}', name='aggregate')
newproj.set_function(func=f'hub://describe:{tag}', name='describe')
newproj.set_function(func=f'hub://feature_selection:{tag}', name="feature_selection")
newproj.set_function(func=f'hub://sklearn_classifier:{tag}', name='train')
newproj.set_function(func=f'hub://test_classifier:{tag}', name='test')
newproj.set_function(func=f'hub://model_server_tester:{tag}', name="model_server-tester")
newproj.set_function(func=f'hub://concept_drift:{tag}', name="concept_drift")
newproj.set_function(func=f'hub://stream_to_parquet:{tag}', name="s2p")
newproj.set_function(func=f'hub://virtual_drift:{tag}', name="virtual_drift")

# Streaming
src_path = os.path.abspath('src/')
newproj.set_function(func=os.path.join(src_path, 'generator.yaml'), name='generator')
newproj.set_function(func=os.path.join(src_path, 'preprocessor.yaml'), name='create_feature_vector')
newproj.set_function(func=os.path.join(src_path, 'inference-server.yaml'), name="serving")
newproj.set_function(func=os.path.join(src_path, 'labeled_stream_creator.yaml'), name="labeled_stream")

## Generate the dataset
If needed go to [Generator](./generator.ipynb) and run the local workflow to generate the metrics dataset to `data/metrics`

## Run the functions locally to develop the workflow

now we can **Run** the function locally on our sample data, we would like to get some details on our `raw` data

## Register raw data as project level artifact

In [7]:
# Define base Dataset
import random
data_dir = os.path.join(os.path.abspath(newproj.context), 'data')
dataset_filename = random.choice(list(filter(lambda x: (x.endswith('pq') or x.endswith('parquet')), os.listdir(data_dir))))
metrics_path = os.path.join(data_dir, dataset_filename)

import pandas as pd
# Drop alternate error columns
label_column = 'is_error'
raw = pd.read_parquet(metrics_path)
raw = raw.drop([col for col in raw.columns if (col != label_column) & (col.endswith(label_column))], axis=1)
dataset_path = os.path.join(data_dir, 'metrics.pq')
raw.to_parquet(dataset_path)

# Add to the project as a Dataset Artifact
from mlrun.artifacts import DatasetArtifact
metrics_artifact = DatasetArtifact(key='metrics', df=raw, format='parquet', target_path=dataset_path)
# newproj.log_artifact(metrics_artifact)
newproj.log_artifact(metrics_artifact)

> 2020-07-28 10:16:44,153 [debug] log artifact metrics at /User/demo-network-operations/data/metrics.pq, size: 171158, db: Y


### Get statistics about the metrics data

In [8]:
from mlrun import NewTask, mount_v3io

In [9]:
describe_task = NewTask(
    name="describe", 
    handler="summarize",  
    params={"key": "summary", 
            "label_column": label_column, 
            'class_labels': ['0', '1'],
            'plot_hist': True,
            'plot_dest': 'plots-metrics'},
    inputs={"table": f'store://{newproj.name}/metrics'},
    artifact_path=ARTIFACT_PATH)

In [10]:
decsribe_run = newproj.func('describe').apply(mount_v3io()).run(describe_task)

> 2020-07-28 10:16:44,969 [debug] Validating field against patterns: {'field_name': 'run.metadata.name', 'field_value': 'describe', 'pattern': ['^.{0,63}$', '^(([A-Za-z0-9][-A-Za-z0-9_.]*)?[A-Za-z0-9])?$']}
> 2020-07-28 10:16:44,977 [info] starting run describe uid=5f2fe96f647d455f9772ebcf9f1a3329  -> http://mlrun-api:8080
> 2020-07-28 10:16:45,154 [info] Job is running in the background, pod: describe-g8w2d
> 2020-07-28 10:16:49,788 [debug] Validating field against patterns: {'field_name': 'run.metadata.name', 'field_value': 'describe', 'pattern': ['^.{0,63}$', '^(([A-Za-z0-9][-A-Za-z0-9_.]*)?[A-Za-z0-9])?$']}
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/requests/adapters.py", line 449, in send
    timeout=timeout
  File "/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py", line 846, in urlopen
    **response_kw
  File "/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py", line 760, in urlopen
    **response_kw
  File "/o

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...9f1a3329,0,Jul 28 10:16:45,error,describe,v3io_user=adminkind=jobowner=admin,table,"key=summarylabel_column=is_errorclass_labels=['0', '1']plot_hist=Trueplot_dest=plots-metrics",,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 5f2fe96f647d455f9772ebcf9f1a3329 --project default , !mlrun logs 5f2fe96f647d455f9772ebcf9f1a3329 --project default
> 2020-07-28 10:17:00,441 [info] run executed, status=error
runtime error: error, check logs


RunError: error, check logs

### Create the feature vector

We will use our [Aggregate](https://github.com/mlrun/functions/blob/master/aggregate/aggregate.ipynb) function to create rolling window features for our feature vector.

In doing so we hope that we could help our algorithms identify local errors by using a windowed trend

In [ ]:
# Define aggregate task
from mlrun import NewTask
aggregate_task = NewTask(
    name='aggregate',
    params={'metrics': ['cpu_utilization', 'throughput', 'packet_loss', 'latency'],
            'metric_aggs': ['mean', 'sum', 'std', 'var', 'min', 'max', 'median'],
            'suffix': 'daily',
            'append_to_df': True,
            'window': 20,
            'center': False,
            'save_to': os.path.join('data', 'aggregate.pq'),
            'drop_na': True},
    inputs={'df_artifact': f'store://{newproj.name}/metrics'},
    handler='aggregate')

In [ ]:
aggregate_run = newproj.func('aggregate').apply(mount_v3io()).run(aggregate_task)

### Get statistics about the feature vector

In [ ]:
aggregate_describe_task = NewTask(
    name="describe-aggregate", 
    handler="summarize",  
    params={"key": "summary", 
            "label_column": label_column, 
            'class_labels': ['0', '1'],
            'plot_hist': True,
            'plot_dest': 'plots-aggregate',
            'sample': 0.3},
    inputs={"table": aggregate_run.outputs['aggregate']},
    artifact_path=ARTIFACT_PATH)

In [ ]:
aggregate_decsribe_run = newproj.func('describe').apply(mount_v3io()).run(aggregate_describe_task)

## Create workflow to train the model
After reviewing the data and creating the feature vector we move to training our model.  
For this task we will use an **LGBM** classifier.  To control the training process we will supply a `model_config` dictionary with the following parameters:
- **CLASS**: Model-specific parameters.
- **FIT**: Training parameters (like epoch when needed)
- **META**: Model and Package version

### Setup model configurations

In [ ]:
model_configs = {
    "CLASS" : {
        "boosting_type"      : "gbdt",
        "num_leaves"         : 300,
        "max_depth"          : 50,
        "learning_rate"      : 0.1,
        "n_estimators"       : 300,
        "objective"          : "binary",
        "scale_pos_weight"   : 1,    
        "min_split_gain"     : 0.0,
        "min_child_samples"  : 20,
        "subsample"          : 1,
        "colsample_bytree"   : 1,
        "reg_alpha"          : 0,
        "reg_lambda"         : 1,
        "n_jobs"             : 16,
        "silent"             : True,
        "importance_type"    : "split",
        "random_state"       : 1},
    "FIT" : {
        "verbose"               : False
    },
    "META" : {
        "class" : "lightgbm.sklearn.LGBMClassifier",
        "version" : "2.3.1"
    }
}
model_config_path = os.path.join(os.path.abspath(newproj.context), 'data', 'lgb_model.json')
with open(model_config_path, 'w') as f:
          f.write(json.dumps(model_configs))

In [ ]:
newproj.log_artifact('lgb_configs',
                     target_path = os.path.abspath(model_config_path))

### Create Pipeline Workflow

In [ ]:
%%writefile src/workflow.py
from kfp import dsl
from mlrun import mount_v3io, mlconf
import os
from nuclio.triggers import V3IOStreamTrigger

funcs = {}
projdir = os.getcwd()
# mlconf.hub_url = 'https://raw.githubusercontent.com/mlrun/functions/{tag}/{name}/function.yaml'
# mlconf.hub_url |= '/User/functions/{name}/function.yaml'
model_inference_stream = '/users/admin/demo-network-operations/streaming/predictions'
model_inference_url = f'http://v3io-webapi:8081{model_inference_stream}'
labeled_stream = 'http://v3io-webapi:8081/users/admin/demo-network-operations/streaming/labeled_stream'

def init_functions(functions: dict, project=None, secrets=None):
    for f in functions.values():
        # Add V3IO Mount
        f.apply(mount_v3io())
        
        # Always pull images to keep updates
        f.spec.image_pull_policy = 'Always'
    
    # Define inference-stream related triggers
    functions['s2p'].add_trigger('labeled_stream', V3IOStreamTrigger(url=f'{labeled_stream}@s2p'))
                
        
@dsl.pipeline(
    name='Network Operations Demo',
    description='Train a Failure Prediction LGBM Model over sensor data'
)
def kfpipeline(
        # aggregate
        df_artifact = '/User/demo-network-operations/data/metrics.pq',
        metrics = ['cpu_utilization', 'throughput', 'packet_loss', 'latency'],
        metric_aggs = ['mean', 'sum', 'std', 'var', 'min', 'max', 'median'],
        suffix = 'daily',
        window = 10,

        # describe
        describe_table = 'netops',
        describe_sample = 0.3,
        label_column = 'is_error',
        class_labels = [1, 0],
        plot_hist = True,
    
        # Feature selection
        k = 5,
        min_votes = 3,
    
        # Train
        sample_size      = -1,        # -n for random sample of n obs, -1 for entire dataset, +n for n consecutive rows
        test_size        = 0.1,       # 10% set aside
        train_val_split  = 0.75,      # remainder split into train and val
    
        # Test
        predictions_col = 'predictions',
    
        # Deploy
        deploy_streaming = True,
        aggregate_fn_url = 'hub://aggregate',
        streaming_features_table = '/User/demo-network-operations/streaming/features',
        streaming_predictions_table = '/User/demo-network-operations/streaming/predictions',
    
        # Streaming
        streaming_metrics_table = '/User/demo-network-operations/streaming/metrics',
    
        # labeled stream creator
        streaming_labeled_table = '/users/admin/demo-network-operations/streaming/labeled_stream',
        
        # Concept drift
        deploy_concept_drift = True,
        secs_to_generate = 10,
        concept_drift_models = ['ddm', 'eddm', 'pagehinkley'],
        output_tsdb = '/User/demo-network-operations/streaming/drift_tsdb',
        input_stream = 'http://v3io-webapi:8081/users/admin/demo-network-operations/streaming/labeled_stream',
        output_stream = '/User/demo-network-operations/streaming/drift_stream',
        streaming_parquet_table =  '/User/demo-network-operations/streaming/inference_pq/',
    
        # Virtual drift
        results_tsdb_table = 'admin/demo_network_operations/streaming/drift_magnitude'
    ):
    
    # Run preprocessing on the data
    aggregate = funcs['aggregate'].as_step(name='aggregate',
                                                  params={'metrics': metrics,
                                                          'metric_aggs': metric_aggs,
                                                          'suffix': suffix,
                                                          'window': window},
                                                  inputs={'df_artifact': df_artifact},
                                                  outputs=['aggregate'],
                                                  handler='aggregate',
                                                  image='mlrun/ml-models')

    describe = funcs['describe'].as_step(name='describe-aggregation',
                                                handler="summarize",  
                                                params={"key": f"{describe_table}_aggregate", 
                                                        "label_column": label_column, 
                                                        'class_labels': class_labels,
                                                        'plot_hist': plot_hist,
                                                        'plot_dest': 'plots/aggregation',
                                                        'sample': describe_sample},
                                                inputs={"table": aggregate.outputs['aggregate']},
                                                outputs=["summary", "scale_pos_weight"])
    
    feature_selection = funcs['feature_selection'].as_step(name='feature_selection',
                                                           handler='feature_selection',
                                                           params={'k': k,
                                                                   'min_votes': min_votes,
                                                                   'label_column': label_column},
                                                           inputs={'df_artifact': aggregate.outputs['aggregate']},
                                                           outputs=['feature_scores', 
                                                                    'max_scaled_scores_feature_scores'
                                                                    'selected_features_count', 
                                                                    'selected_features'],
                                                           image='mlrun/ml-models')
    
    describe = funcs['describe'].as_step(name='describe-feature-vector',
                                            handler="summarize",  
                                            params={"key": f'{describe_table}_features', 
                                                    "label_column": label_column, 
                                                    'class_labels': class_labels,
                                                    'plot_hist': plot_hist,
                                                    'plot_dest': 'plots/feature_vector'},
                                            inputs={"table": feature_selection.outputs['selected_features']},
                                            outputs=["summary", "scale_pos_weight"])
    
    train = funcs['train'].as_step(name='train',
                                   params={"sample"          : sample_size, 
                                           "label_column"    : label_column,
                                           "test_size"       : test_size,
                                           "train_val_split" : train_val_split},
                                   inputs={"dataset"         : feature_selection.outputs['selected_features']},
                                   hyperparams={'model_pkg_class': ["sklearn.ensemble.RandomForestClassifier", 
                                                                    "sklearn.linear_model.LogisticRegression",
                                                                    "sklearn.ensemble.AdaBoostClassifier"]},
                                   selector='max.accuracy',
                                   outputs=['model', 'test_set'],
                                   image='mlrun/ml-models')
    
    test = funcs['test'].as_step(name='test',
                                 handler='test_classifier',
                                 params={'label_column': label_column,
                                         'predictions_column': predictions_col},
                                 inputs={'models_path': train.outputs['model'],
                                         'test_set': train.outputs['test_set']},
                                 outputs=['test_set_preds'],
                                 image='mlrun/ml-models')

    
    with dsl.Condition(deploy_streaming == True):
        
        # deploy the model using nuclio functions
        deploy = funcs['serving'].deploy_step(env={'model_path': train.outputs['model'],
                                                   'FEATURES_TABLE': streaming_features_table,
                                                   'PREDICTIONS_TABLE': streaming_predictions_table,
                                                   'prediction_col': predictions_col}, 
                                              tag='v1')

        # test out new model server (via REST API calls)
        tester = funcs["model_server-tester"].as_step(name='model-tester',
                                                      params={'addr': deploy.outputs['endpoint'], 
                                                              'model': "predictor",
                                                              'label_column': label_column},
                                                      inputs={'table': train.outputs['test_set']},
                                                      outputs=['test_set_preds'])
    
        # Streaming demo functions
        preprocessor = funcs['create_feature_vector'].deploy_step(env={ 'aggregate_fn_url': aggregate_fn_url,
                                                                'METRICS_TABLE': streaming_metrics_table,
                                                                'FEATURES_TABLE': streaming_features_table,
                                                                'metrics': metrics,
                                                                'metric_aggs': metric_aggs,
                                                                'suffix': suffix,
                                                                'base_dataset': train.outputs['test_set'],
                                                                'label_col': label_column}).after(tester)

        labeled_stream_creator = funcs['labeled_stream'].deploy_step(env={'METRICS_TABLE': streaming_metrics_table,
                                                                                  'PREDICTIONS_TABLE': streaming_predictions_table,
                                                                                  'OUTPUT_STREAM': streaming_labeled_table,
                                                                                  'label_col': label_column,
                                                                                  'prediction_col': predictions_col}).after(tester)

        generator = funcs['generator'].deploy_step(env={'SAVE_TO': streaming_metrics_table,
                                                        'SECS_TO_GENERATE': secs_to_generate}).after(preprocessor)
        
        with dsl.Condition(deploy_concept_drift == True):

            concept_builder = funcs['concept_drift'].deploy_step(skip_deployed=True)

            concept_drift = funcs['concept_drift'].as_step(name='concept_drift_deployer',
                                                           params={'models': concept_drift_models,
                                                                   'label_col': label_column,
                                                                   'prediction_col': predictions_col,
                                                                   'output_tsdb': output_tsdb,
                                                                   'input_stream': f'{input_stream}@cds',
                                                                   'output_stream': output_stream},
                                                           inputs={'base_dataset': test.outputs['test_set_preds']},
                                                           artifact_path=mlconf.artifact_path,
                                                           image=concept_builder.outputs['image']).after(labeled_stream_creator)

            s2p = funcs['s2p'].deploy_step(env={'window': 10,
                                                'features': metrics,
                                                'save_to': streaming_parquet_table,
                                                'base_dataset': test.outputs['test_set_preds'],
                                                'results_tsdb_container': 'users',
                                                'results_tsdb_table': results_tsdb_table,
                                                'mount_path': '/users/admin',
                                                'mount_remote': '/User'}).after(tester)
    

## Add workflow

In [ ]:
newproj.set_workflow('main', os.path.join(os.path.abspath(newproj.context), 'src', 'workflow.py'))

## Save Project

In [ ]:
newproj.save(os.path.join(newproj.context, 'project.yaml'))

### Run workflow

In [ ]:
newproj.run('main', artifact_path=ARTIFACT_PATH, dirty=True)

In [38]:
pd.read_parquet('streaming/predictions/20200728T021224-20200728T021234.parquet')

cpu_utilization  \
timestamp               company                   data_center    device                           
2020-07-28 02:12:24.670 Sanchez__Nunez_and_Fuller Kristin_Road   8452501176692        78.648990   
                                                                 9774835914273        63.229630   
                        Richardson_Group          Michael_Fords  8707984323649        59.795900   
                                                                 0985634152997        83.660315   
                                                  Moody_Causeway 9660818354208        63.825377   
                                                                 8631608180722        64.600709   
2020-07-28 02:12:29.670 Sanchez__Nunez_and_Fuller Brown_Camp     8181800417886        56.486986   
                                                                 9119677190168        67.812418   
                                                  Kristin_Road   8452501176692        60.219226   
                                                                 9774835914273        67.790305   
                        Richardson_Group          Michael_Fords  8707984323649        54.248583   
                                                                 0985634152997        71.333091   
                                                  Moody_Causeway 9660818354208        60.440096   
                                                                 8631608180722        94.538691   
2020-07-28 02:12:34.670 Sanchez__Nunez_and_Fuller Brown_Camp     8181800417886        69.516329   
                                                                 9119677190168        67.599632   
                                                  Kristin_Road   8452501176692        59.629267   
                                                                 9774835914273        55.899586   
                        Richardson_Group          Michael_Fords  8707984323649        70.881429   
                                                                 0985634152997        72.706430   
                                                  Moody_Causeway 9660818354208        59.613381   
                                                                 8631608180722        68.341236   

                                                                                 latency  \
timestamp               company                   data_center    device                    
2020-07-28 02:12:24.670 Sanchez__Nunez_and_Fuller Kristin_Road   8452501176692  9.671623   
                                                                 9774835914273  8.393025   
                        Richardson_Group          Michael_Fords  8707984323649  0.000000   
                                                                 0985634152997  0.000000   
                                                  Moody_Causeway 9660818354208  0.000000   
                                                                 8631608180722  2.238899   
2020-07-28 02:12:29.670 Sanchez__Nunez_and_Fuller Brown_Camp     8181800417886  8.232113   
                                                                 9119677190168  0.000000   
                                                  Kristin_Road   8452501176692  0.000000   
                                                                 9774835914273  0.000000   
                        Richardson_Group          Michael_Fords  8707984323649  0.000000   
                                                                 0985634152997  2.782231   
                                                  Moody_Causeway 9660818354208  0.000000   
                                                                 8631608180722  1.648133   
2020-07-28 02:12:34.670 Sanchez__Nunez_and_Fuller Brown_Camp     8181800417886  0.000000   
                                                                 9119677190168  2.305762   
                                               